In [2]:
#!python -m pip install rasa

Add intents into the dataset

In [7]:
nlu_content = """
version: "3.1"

nlu:
- intent: greet
  examples: |
    - Hi
    - Hello
    - Hey bot
    - What's up?
    - Sup
    - Yo

- intent: show_todos
  examples: |
    - Show my tasks
    - What's on my list?
    - What do I need to do?
    - List my todos

- intent: thank
  examples: |
    - Thanks
    - Thank you
    - Appreciate it
    - You're the best

- intent: goodbye
  examples: |
    - Bye
    - Goodbye
    - See you later
    - Later

- intent: add_todo
  examples: |
    - Add reminder to [call mom]
    - Add task [buy groceries]
    - Remind me to [water plants]
    - I need to [finish report]
    - Create new reminder to [schedule meeting]
    - Create reminder [pay rent]
    - Make a task to [book flight]
    - Add [buy milk] to my list
    - Remind me to [call mom]
    - I need to [finish homework]
    - Can you add [schedule meeting]?

- intent: delete_todo
  examples: |
    - Delete task [buy milk]
    - Remove reminder [call boss]
    - Cancel [write essay]
    - Void [clean room]

- intent: clear_all_todos
  examples: |
    - Erase my entire to do list
    - Void my entire to do list
    - Self-destruct my entire to do list
    - Cancel my entire to do list
    - Delete list
    - Delete to-do list
    - Erase list

- intent: request_motivation
  examples: |
    - Give motivation
    - Motivate me
    - I need encouragement
    - Say something inspiring
    - Pep talk please
    - I don't want to [clean my room]
"""

with open("data/nlu.yml", "w") as f:
    f.write(nlu_content)

In [13]:
domain_content = """

version: "3.1"

intents:
  - greet
  - add_todo
  - show_todos
  - thank
  - goodbye
  - delete_todo
  - clear_all_todos
  - request_motivation

responses:
  utter_greet:
  - text: "Oh great, another human. What do you want?"
  - text: "You again? Fine, what’s up?"
  - text: "What do you want now?"

  utter_thanks:
  - text: "Wow, gratitude. How original."
  - text: "You're welcome, I guess."

  utter_goodbye:
  - text: "Finally. Go away."
  - text: "Don’t let the door hit you."

  utter_ask_task:
  - text: "What’s the task? Or are you too busy being indecisive?"

  utter_show_todos:
  - text: "Here’s your list (you’re so disorganized):
- {todos}"

  utter_priority_confirm:
  - text: "Sure, I'll pretend this is '{priority}' priority. *eye roll*"

  utter_motivation:
  - text: "Wow, you actually want to be productive? Miracles do happen."
  - text: "If I had a dollar for every time you asked for motivation... oh wait, I'm a bot."

  utter_all_todos_cleared:
  - text: "Poof! Your list is gone. Was it that overwhelming?"
  - text: "List deleted. Now you can procrastinate guilt-free!"

  utter_goodbye_with_tasks:
  - text: "Leaving already? You still have {count} tasks pending. Typical."
  - text: "Quitting? Shocking. You still had {count} things to do."

  utter_goodbye_all_done:
  - text: "OMG, you finished everything. I didn’t think you could do it."
  - text: "See you never."
  
entities:
  - task
  - priority

slots:
  priority:
    type: categorical
    mappings:
    - type: from_entity
      entity: priority
    values:
      - ASAP
      - Whenever
      - Backburner
  todo_list:
    type: list
    mappings:
    - type: custom
    influence_conversation: false

actions:
  - action_add_todo
  - action_show_todos
  - action_delete_todo
  - action_clear_all_todos
  - action_check_pending_tasks
  - action_give_motivation
"""

with open("domain.yml", "w", encoding="utf-8") as f:
    f.write(domain_content)

In [14]:
rules_content = """
version: "3.1"

rules:
  - rule: Greeting
    steps:
      - intent: greet
      - action: utter_greet

  - rule: Add Todo
    steps:
      - intent: add_todo
      - action: action_add_todo

  - rule: Show Todos
    steps:
      - intent: show_todos
      - action: action_show_todos

  - rule: Delete Todo
    steps:
      - intent: delete_todo
      - action: action_delete_todo

  - rule: Clear All Todos
    steps:
      - intent: clear_all_todos
      - action: action_clear_all_todos

  - rule: Request Motivation
    steps:
      - intent: request_motivation
      - action: action_give_motivation

  - rule: Thank Response
    steps:
      - intent: thank
      - action: utter_thanks
      
  - rule: Goodbye
    steps:
      - intent: goodbye
      - action: action_check_pending_tasks  # Will handle both empty/non-empty cases
  # - rule: Goodbye (no tasks)
  #   condition:
  #     - slot_was_set:
  #         - todo_list: []
  #   steps:
  #     - intent: goodbye
  #     - action: utter_goodbye_all_done

  # - rule: Goodbye (with tasks)
  #   condition:
  #     - slot_was_set:
  #         - todo_list: 
  #     - not: 
  #         slot_was_set:
  #         - todo_list: []
  #     - active_loop: null
  #   steps:
  #     - intent: goodbye
  #     - action: action_check_pending_tasks
"""

with open("data/rules.yml", "w") as f:
    f.write(rules_content)